## Visualisation of backtesting

In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pylab import plt
import mplfinance as mpf
#from talib.Momentum import AwesomeOscillatorIndicator
import talib as ta
# from ta.momentum import AwesomeOscillatorIndicator
# from zigzag import peak_valley_pivots
"""
for interactive plots (first - pip3 install ipywidgets)
1. pip3 install ipywidgets
for jupyter lab
2. jupyter labextension install @jupyter-widgets/jupyterlab-manager
or for jupyter notebook
2. !jupyter nbextension enable --py widgetsnbextension
"""

'\nfor interactive plots (first - pip3 install ipywidgets)\n1. pip3 install ipywidgets\nfor jupyter lab\n2. jupyter labextension install @jupyter-widgets/jupyterlab-manager\nor for jupyter notebook\n2. !jupyter nbextension enable --py widgetsnbextension\n'

In [2]:
data = pd.read_csv('backtesting.csv')

In [3]:
data = data.rename(columns={'open': 'Open', 'high': 'High', 'low':'Low', 'close':'Close'})
data['d_t'] = data['date'] + ' ' + data['time']
data.index = pd.DatetimeIndex(data.d_t)
data.index.name = 'Date_Time'
data = data.drop(columns=['time', 'Unnamed: 0', 'date', 'd_t'])
data = data.dropna()

In [4]:
#data[50:55]
print(data.shape)
data.head()

(2531, 7)


,Open,High,Low,Close,action,P/L,Total profit
Date_Time,,,,,,,
2020-01-03 11:00:00,1.30976,1.31007,1.30662,1.30676,hold,0.0,0.0
2020-01-03 12:00:00,1.30676,1.30878,1.30676,1.30778,hold,0.0,0.0
2020-01-03 13:00:00,1.30778,1.30784,1.30532,1.30581,hold,0.0,0.0
2020-01-03 14:00:00,1.30581,1.30821,1.30537,1.30798,hold,0.0,0.0
2020-01-03 15:00:00,1.30798,1.31096,1.30718,1.30723,hold,0.0,0.0


In [16]:
def buy_sell(data, close_data):

    buy_signal, sell_signal = [], []
    for date, value in data.iteritems():
        #print(value)
        if (value == 'buy'):        #value == '1'
            buy_signal.append((close_data[date]*0.999))
            sell_signal.append(np.nan)
        elif (value == 'sell'):     #value == '2'
            #sell.append((date, (close_data[date]+1)))
            sell_signal.append(close_data[date]*1.001)
            buy_signal.append(np.nan)
        else:
            buy_signal.append(np.nan)
            sell_signal.append(np.nan)
 
    return buy_signal, sell_signal
#b, s = buy_sell(data['action'][-200:], data['Close'][-200:])

In [13]:
# pd.notna(b).any()

True

In [19]:

@interact
def interactive_plot(column='close', past_n=1000, datapoints=500):

    past_n_n = past_n-datapoints
    
    b, s = buy_sell(data['action'][-past_n:-past_n_n], data['Close'][-past_n:-past_n_n])
    
    buy = mpf.make_addplot(b, type='scatter', marker='^', markersize=100, color='blue')
    sell = mpf.make_addplot(s, type='scatter', marker='v', markersize=100, color='orange')
    
    apds = []
    if (pd.notna(b).any() and pd.notna(s).any()):
        apds = [buy, sell]
    elif(pd.notna(b).any()):
        apds = [buy]
    elif(pd.notna(s).any()):
        apds = [sell]

    mpf.plot(data[-past_n:-past_n_n], figsize=(20,10), type='candle', style='charles', title='', ylabel='', ylabel_lower='', addplot=apds) 

interactive(children=(Text(value='close', description='column'), IntSlider(value=1000, description='past_n', m…